<a href="https://colab.research.google.com/github/MarinaWolters/Coding-Tracker/blob/master/Hands_on3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Representation Hands-on

A lot of data is not just in a "flat file" format such as .csv, but may be in a hierarchical format such as .json.  Since much of our analysis is done on tables,  how can we go between hierarchical and tabular data?  

We'll look at:

1. Hierarchical data
2. the explode() function
3. groupby with lists
4. merge (join) to reassemble results or traverse relationships


We'll start with data from LinkedIn!

In [ ]:
import pandas as pd
import numpy as np

# JSON parsing
import json

# SQLite RDBMS
import sqlite3

# Time conversions
import time


In [ ]:
!wget -nc https://storage.googleapis.com/penn-cis5450/linkedin_small.json

--2023-02-02 21:37:14--  https://storage.googleapis.com/penn-cis5450/linkedin_small.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 173.194.210.128, 173.194.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480593111 (458M) [application/json]
Saving to: ‘linkedin_small.json’

linkedin_small.json 100%[===================>] 458.33M   139MB/s    in 3.5s    

2023-02-02 21:37:18 (132 MB/s) - ‘linkedin_small.json’ saved [480593111/480593111]



In [ ]:
!ls /content

linkedin_small.json  sample_data


In [ ]:
!head /content/linkedin_small.json

{ "_id" : "in-00000001", "name" : { "family_name" : "Mazalu MBA", "given_name" : "Dr Catalin" }, "locality" : "United States", "skills" : [ "Key Account Development", "Strategic Planning", "Market Planning", "Team Leadership", "Negotiation", "Forecasting", "Key Account Management", "Sales Management", "New Business Development", "Business Planning", "Cross-functional Team Leadership", "Budgeting", "Strategy Development", "Business Strategy", "Consultative Selling", "Medical Devices", "Customer Relations", "Contract Negotiation", "Mentoring", "Coaching", "Healthcare", "Territory", "Sales Process", "Direct Sales", "Sales Operations", "Pharmaceutical Sales" ], "industry" : "Medical Devices", "summary" : "SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJECT MANAGEMENTDOMESTIC & INTERNATIONAL KEY ACCOUNT MANAGEMENTBusiness and Sales Executive with 20 years of accomplished career track, reflecting extensive experience and dynamic record-breaking performance in the Medical Industry markets. Exce

## 1. Let's load the data

We'll pull in the JSON data, parse it, and create a dataframe.

Take a look at what's nested.

In [ ]:
%%time
# 100K records from linkedin
linked_in = open('/content/linkedin_small.json')
    
people = []

for line in linked_in:
  person = json.loads(line)
  people.append(person)
    
people_df = pd.DataFrame(people)

display(people_df)

,_id,name,locality,skills,industry,summary,url,also_view,education,group,overview_html,interval,experience,specilities,events,interests,homepage,honors
0,in-00000001,"{'family_name': 'Mazalu MBA', 'given_name': 'D...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,http://www.linkedin.com/in/00000001,[{'url': 'http://www.linkedin.com/pub/krisa-dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,in-00001,"{'family_name': 'Forslund', 'given_name': 'Ann'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,http://be.linkedin.com/in/00001,[{'url': 'http://www.linkedin.com/pub/peter-ki...,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...","<dl id=""overview""><dt id=""overview-summary-cur...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",NaN,NaN,NaN
2,in-00006,"{'family_name': 'Douglas', 'given_name': 'Shawn'}","San Francisco, California","[DNA, Nanotechnology, Molecular Biology, Softw...",Research,I am interested in inventing new methods to co...,http://www.linkedin.com/in/00006,[{'url': 'http://www.linkedin.com/pub/george-c...,"[{'major': 'Biophysics', 'end': '2009', 'name'...",NaN,"<dl id=""overview""><dt id=""overview-summary-cur...",0.0,"[{'org': 'UCSF', 'title': 'Assistant Professor...",NaN,[{'from': 'Wyss Institute for Biologically Ins...,"personal genomics, nanotechnology","{'BIOMOD': ['http://biomod.net/'], 'Company We...",NaN
3,in-000montgomery,"{'family_name': 'Kilimann', 'given_name': 'Edr...",San Francisco Bay Area,NaN,Information Technology and Services,OBJECTIVE<Primary> Work on an interesting and ...,http://www.linkedin.com/in/000montgomery,[{'url': 'http://www.linkedin.com/pub/david-br...,NaN,"{'affilition': ['Big Data, Low Latency', 'Expe...",NaN,5.0,"[{'org': '<Online Recruiting Company>', 'desc'...",NaN,"[{'from': '<Employee Benefits, Administration ...",NaN,NaN,NaN
4,in-000vijaychauhan,"{'family_name': 'Chauhan, PMP', 'given_name': ...","Chennai Area, India","[Program Management, French, Avionics, Embedde...",Aviation & Aerospace,"Experience in Avionics Systems, Embedded Syste...",http://in.linkedin.com/in/000vijaychauhan,[{'url': 'http://in.linkedin.com/in/sandeeprag...,"[{'start': '1988', 'end': '1989', 'name': 'Eco...",{'member': 'Member of Project Management Insti...,NaN,NaN,NaN,NaN,NaN,"Literature, Philosophy, Music",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,in-dorothyballarini,"{'family_name': 'Ballarini', 'given_name': 'Do...","London, United Kingdom","[Zbrush, 3D Studio Max, Concept Design, Charac...",Motion Pictures and Film,I have 10 years of experience working with the...,http://uk.linkedin.com/in/dorothyballarini,[{'url': 'http://es.linkedin.com/pub/rebeca-pu...,"[{'start': '2007', 'major': 'Design', 'end': '...","{'affilition': ['3D Animation, VFX & Games', '...",NaN,6.0,"[{'org': 'ZOO', 'title': 'Freelancer 3D artist...",NaN,[{'from': 'Universidade Federal do Rio Grande ...,"Arts, Computer Graphics, Cinema, Video Games",NaN,NaN
99996,in-dorothybarnettgrimes,"{'family_name': 'Grimes', 'given_name': 'Dorot...","Houston, Texas Area","[Internal Controls, Revenue Recognition, Sarba...",Oil & Energy,• Results driven finance leader and business p...,http://www.linkedin.com/in/dorothybarnettgrimes,[{'url': 'http://www.linkedin.com/pub/ken-greg...,"[{'major': 'Business', 'name': 'Santa Clara Un...",NaN,NaN,25.0,"[{'org': 'Spectrum ASA', 'title': 'Multi-Clien...",NaN,"[{'from': 'TanThap Inc and Digitan Systems', '...",NaN,NaN,[Certified Public Accountant - State of Texas]
99997,in-dorothycarroll,"{'family_name': 'Huffman', 'given_name': 'Doro...","Richmond, Virginia Area",NaN,Information Technology and Services,•Over 6 years experience in all ph

CPU times: user 9.03 s, sys: 1.94 s, total: 11 s
Wall time: 10.9 s


### 1.1. Extract names

Note that we have a column called `name` that has nested key-value pairs.  How might we turn this into a table?

Write code to create `people_df2` that appropriately includes first and last names, *without* using nesting.

In [ ]:
people_df2 = people_df#.copy()

### TODO 1:  Update people_df2 so it has the family and given names in non-nested form.  
###          Drop the name column. 
people_df2[['name']]

,name
0,"{'family_name': 'Mazalu MBA', 'given_name': 'D..."
1,"{'family_name': 'Forslund', 'given_name': 'Ann'}"
2,"{'family_name': 'Douglas', 'given_name': 'Shawn'}"
3,"{'family_name': 'Kilimann', 'given_name': 'Edr..."
4,"{'family_name': 'Chauhan, PMP', 'given_name': ..."
...,...
99995,"{'family_name': 'Ballarini', 'given_name': 'Do..."
99996,"{'family_name': 'Grimes', 'given_name': 'Dorot..."
99997,"{'family_name': 'Huffman', 'given_name': 'Doro..."
99998,"{'family_name': 'Czudziak', 'given_name': 'Dor..."


In [ ]:
def f(name) :
  return name['family_name']
people_df2['family_name'] = people_df2['name'].apply(f)

In [ ]:
people_df2['family_name']

0          Mazalu MBA
1            Forslund
2             Douglas
3            Kilimann
4        Chauhan, PMP
             ...     
99995       Ballarini
99996          Grimes
99997         Huffman
99998        Czudziak
99999          Dalton
Name: family_name, Length: 100000, dtype: object

In [ ]:
people_df2['family_name'] = people_df2['name'].apply(lambda name: name['family_name'])

### 1.2 Skills

Note that we have another element, skills, which is a list of values (strings).  Since we have an unbounded number of skills for each person, we need to split this into a separate table containing the _id of the person and one of their skills.  For each person, there will be a separate row for each of their skills.

To do this we can use a function in Pandas called `explode`, which will create multiple rows, one for each item in the list in the exploded column.  It's super convenient!

In [ ]:
people_df2['skills']

0        [Key Account Development, Strategic Planning, ...
1                          [Molecular Biology, Biomarkers]
2        [DNA, Nanotechnology, Molecular Biology, Softw...
3                                                      NaN
4        [Program Management, French, Avionics, Embedde...
                               ...                        
99995    [Zbrush, 3D Studio Max, Concept Design, Charac...
99996    [Internal Controls, Revenue Recognition, Sarba...
99997                                                  NaN
99998                                                  NaN
99999    [Candidate Generation, Candidate Development, ...
Name: skills, Length: 100000, dtype: object

In [ ]:
## TODO 2:
## Using explode, extract the skills for each person and put in a dataframe skills_df.  
# Remove skills from people_df2.

skills_df = people_df2[['_id', 'skills']].explode('skills')
display(skills_df)
#display(people_df2)

,_id,skills
0,in-00000001,Key Account Development
0,in-00000001,Strategic Planning
0,in-00000001,Market Planning
0,in-00000001,Team Leadership
0,in-00000001,Negotiation
...,...,...
99999,in-dorothydalton,Staff Development
99999,in-dorothydalton,Employee Wellness
99999,in-dorothydalton,Personnel Management
99999,in-dorothydalton,Sourcing


In [ ]:
people_df2 = people_df2.drop(columns='skills')

In [ ]:
### TODO 3: To show that we can go back and forth between these two representations, 
###         let's reconstruct people and their skills as a list in a dataframe skills_list_df.
###         Use groupby, then apply(list) to the skills.

skills_df.groupby('_id')['skills']

In [ ]:
for i in skills_df.groupby('_id')['skills']:
  print(i)
  print(type(i[1]))
  break

('in-00000001', 0             Key Account Development
0                  Strategic Planning
0                     Market Planning
0                     Team Leadership
0                         Negotiation
0                         Forecasting
0              Key Account Management
0                    Sales Management
0            New Business Development
0                   Business Planning
0    Cross-functional Team Leadership
0                           Budgeting
0                Strategy Development
0                   Business Strategy
0                Consultative Selling
0                     Medical Devices
0                  Customer Relations
0                Contract Negotiation
0                           Mentoring
0                            Coaching
0                          Healthcare
0                           Territory
0                       Sales Process
0                        Direct Sales
0                    Sales Operations
0                Pharmaceutical Sa

In [ ]:
def transform_to_list(items):
  return list(items)
skills_df.groupby('_id')['skills'].apply(transform_to_list)

In [ ]:
skills_list_df = skills_df.groupby('_id')['skills'].apply(list).reset_index()

In [ ]:
type(skills_list_df)

pandas.core.series.Series

In [ ]:
skills_list_df

,_id,skills
0,in-00000001,"[Key Account Development, Strategic Planning, ..."
1,in-00001,"[Molecular Biology, Biomarkers]"
2,in-00006,"[DNA, Nanotechnology, Molecular Biology, Softw..."
3,in-000montgomery,[nan]
4,in-000vijaychauhan,"[Program Management, French, Avionics, Embedde..."
...,...,...
99995,in-dorothyballarini,"[Zbrush, 3D Studio Max, Concept Design, Charac..."
99996,in-dorothybarnettgrimes,"[Internal Controls, Revenue Recognition, Sarba..."
99997,in-dorothycarroll,[nan]
99998,in-dorothyczudziak,[nan]


In [ ]:
type(skills_list_df)

pandas.core.frame.DataFrame

## 1.3 Experiences

Can we generalize this idea to experiences?  Again, we can explode.  However in this case each experience is itself a dictionary and needs to be converted to a tuple...

In [ ]:
people_df2[['_id', 'experience']]

,_id,experience
0,in-00000001,NaN
1,in-00001,"[{'org': 'Johnson and Johnson', 'title': 'Seni..."
2,in-00006,"[{'org': 'UCSF', 'title': 'Assistant Professor..."
3,in-000montgomery,"[{'org': '<Online Recruiting Company>', 'desc'..."
4,in-000vijaychauhan,NaN
...,...,...
99995,in-dorothyballarini,"[{'org': 'ZOO', 'title': 'Freelancer 3D artist..."
99996,in-dorothybarnettgrimes,"[{'org': 'Spectrum ASA', 'title': 'Multi-Clien..."
99997,in-dorothycarroll,"[{'org': 'Estes Express Lines', 'title': 'QA L..."
99998,in-dorothyczudziak,"[{'start': 'January 2000', 'desc': 'I'm a full..."


In [ ]:
people_df2[['_id','experience']].explode('experience')

,_id,experience
0,in-00000001,NaN
1,in-00001,"{'org': 'Johnson and Johnson', 'title': 'Senio..."
1,in-00001,"{'org': 'Albert Einstein Medical Center', 'tit..."
1,in-00001,"{'org': 'Columbia University', 'title': 'Assoc..."
1,in-00001,{'org': 'Memorial Sloan Kettering Cancer Cente...
...,...,...
99997,in-dorothycarroll,"{'org': 'Circuit City', 'title': 'Systems Anal..."
99997,in-dorothycarroll,"{'org': 'Circuit City', 'title': 'Programmer/A..."
99998,in-dorothyczudziak,"{'start': 'January 2000', 'desc': 'I'm a full ..."
99998,in-dorothyczudziak,"{'org': 'Arirang Hibachi Steakhouse', 'start':..."


In [ ]:
## TODO 4: explode the experience into experience_df, and drop from original table

experience_df = people_df2[['_id','experience']].explode('experience')

In [ ]:
### TODO 5: now use apply() to create a dictionary for each row in experience_df
###       then convert this (a Series) to a list via to_list()
###       then create a DataFrame.  This will take a very long time!
### Note that the keys in experience are (org, title, start, end), but that some 
### rows may only have a subset of them
def f(row):
  return {'_id': row['_id'], \
          'org': row['experience']['org'] if 'experience' in row and 'org' in row['experience'] else '', \
          'title': row['experience']['title'] if 'experience' in row and 'title' in row['experience'] else '' }

experience_df = \
  pd.DataFrame(
      # TODO 
      experience_df.dropna().apply(f, axis = 1)
  )

experience_df

In [ ]:
def f(row):
  return {'_id': row['_id'], \
          'org': row['experience']['org'] if 'experience' in row and 'org' in row['experience'] else '', \
          'title': row['experience']['title'] if 'experience' in row and 'title' in row['experience'] else '' }

experience_df.dropna().apply(f, axis = 1)

1        {'_id': 'in-00001', 'org': 'Johnson and Johnso...
1        {'_id': 'in-00001', 'org': 'Albert Einstein Me...
1        {'_id': 'in-00001', 'org': 'Columbia Universit...
1        {'_id': 'in-00001', 'org': 'Memorial Sloan Ket...
1        {'_id': 'in-00001', 'org': 'Sahlgrenska Univer...
                               ...                        
99997    {'_id': 'in-dorothycarroll', 'org': 'Capital O...
99997    {'_id': 'in-dorothycarroll', 'org': 'Circuit C...
99997    {'_id': 'in-dorothycarroll', 'org': 'Circuit C...
99998    {'_id': 'in-dorothyczudziak', 'org': '', 'titl...
99998    {'_id': 'in-dorothyczudziak', 'org': 'Arirang ...
Length: 470715, dtype: object

In [ ]:
pd.DataFrame(experience_df.dropna().apply(f, axis = 1).to_list())

,_id,org,title
0,in-00001,Johnson and Johnson,"Senior Scientist, Oncology Biomarkers"
1,in-00001,Albert Einstein Medical Center,Associate at Dept of Molecular Genetics
2,in-00001,Columbia University,Associate Research Scientist
3,in-00001,Memorial Sloan Kettering Cancer Center,Post Doctoral Research Fellow
4,in-00001,Sahlgrenska University Hospital,Research Scientist
...,...,...,...
470710,in-dorothycarroll,Capital One,QA Metrics Analyst - contractor
470711,in-dorothycarroll,Circuit City,Systems Analyst
470712,in-dorothycarroll,Circuit City,Programmer/Analyst
470713,in-dorothyczudziak,,Model


In [ ]:
people_df2

In [ ]:
### TODO 6: Reconstruct people and their skills, with one skill per row.  
### Use people_df2 (_id, family_name, given_name).  Include all people, even
### those with no skills.
people_df2.merge(skills_df, on=['_id'], how='left')


,_id,name,locality,industry,summary,url,also_view,education,group,overview_html,interval,specilities,events,interests,homepage,honors,family_name,skills
0,in-00000001,"{'family_name': 'Mazalu MBA', 'given_name': 'D...",United States,Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,http://www.linkedin.com/in/00000001,[{'url': 'http://www.linkedin.com/pub/krisa-dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mazalu MBA,Key Account Development
1,in-00000001,"{'family_name': 'Mazalu MBA', 'given_name': 'D...",United States,Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,http://www.linkedin.com/in/00000001,[{'url': 'http://www.linkedin.com/pub/krisa-dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mazalu MBA,Strategic Planning
2,in-00000001,"{'family_name': 'Mazalu MBA', 'given_name': 'D...",United States,Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,http://www.linkedin.com/in/00000001,[{'url': 'http://www.linkedin.com/pub/krisa-dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mazalu MBA,Market Planning
3,in-00000001,"{'family_name': 'Mazalu MBA', 'given_name': 'D...",United States,Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,http://www.linkedin.com/in/00000001,[{'url': 'http://www.linkedin.com/pub/krisa-dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mazalu MBA,Team Leadership
4,in-00000001,"{'family_name': 'Mazalu MBA', 'given_name': 'D...",United States,Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,http://www.linkedin.com/in/00000001,[{'url': 'http://www.linkedin.com/pub/krisa-dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mazalu MBA,Negotiation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104668,in-dorothydalton,"{'family_name': 'Dalton', 'given_name': 'Dorot...","Brussels Area, Belgium",Human Resources,NaN,http://be.linkedin.com/in/dorothydalton,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dalton,Staff Development
1104669,in-dorothydalton,"{'family_name': 'Dalton', 'given_name': 'Dorot...","Brussels Area, Belgium",Human Resources,NaN,http://be.linkedin.com/in/dorothydalton,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dalton,Employee Wellness
1104670,in-dorothydalton,"{'family_name': 'Dalton', 'given_name': 'Dorot...","Brussels Area, Belgium",Human Resources,NaN,http://be.linkedin.com/in/dorothydalton,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dalton,Personnel Management
1104671,in-dorothydalton,"{'family_name': 'Dalton', 'given_name': 'Dorot...","Brussels Area, Belgium",Human Resources,NaN,http://be.linkedin.com/in/dorothydalton,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dalton,Sourcing


In [ ]:
### TODO 7: people_df2 (_id, family_name, given_name), their skills (as a list, recall skills_list_df),
###         and their experience.  There will be one row for each job experience for a person, but each row
###         will contain a list of experiences.  


In [ ]:
### TODO 8: IDs of people with the same employer! (Include self-matches)

same_emp = # TODO